In [ ]:
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-4b7e6174-6085-2ce2-a065-eb46aa30a9fd)


# Environment preparation

Install the required packages for the current environment used in Jupyter Notebook

In [ ]:
!pip install transformers==4.24.0 ftfy open_clip_torch==2.7.0 -qqq

     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 1.4 MB 67.4 MB/s 
     |████████████████████████████████| 7.6 MB 64.8 MB/s 
     |████████████████████████████████| 182 kB 93.9 MB/s 


Clone the GitHub repository into current working directory
```
./
├── OFA/
└── ...
```

In [ ]:
!git clone https://github.com/25icecreamflavors/OFA

Cloning into 'OFA'...
remote: Enumerating objects: 6599, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 6599 (delta 105), reused 114 (delta 50), pack-reused 6381
Receiving objects: 100% (6599/6599), 122.85 MiB | 56.51 MiB/s, done.
Resolving deltas: 100% (2709/2709), done.


Install anaconda to create a virtual environment

In [ ]:
# install conda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!sh ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda update --channel defaults --all --yes

--2022-12-19 21:12:16--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  73.06M   165MB/s    in 0.4s    

2022-12-19 21:12:16 (165 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [76607678/76607678]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py39h06a4308_2
    - cffi==1.15.0=py39hd667e15_1
    - charset-normalize

Create conda virtual environment and install required packages

In [ ]:
!conda create -y --name ofa python=3.7.4 pytorch=1.8.1
!source activate ofa && pip install pip==21.2.4 -qqq
!source activate ofa && pip install torchvision==0.9.1 -qqq
!source activate ofa && pip install opencv-python timm ftfy==6.0.3 tensorboardX==2.4.1 pycocotools==2.0.4 pycocoevalcap==1.2 pytorch_lightning -qqq
!source activate ofa && pip install einops datasets rouge_score soundfile editdistance librosa python-Levenshtein zhconv -qqq
!source activate ofa && pip install pypinyin g2p_en -qqq
!source activate ofa && pip install protobuf~=3.20.0 -qqq
!source activate ofa && pip install -U torchaudio --no-cache-dir
!source activate ofa && cd OFA && pip install -e ./fairseq/

Solving environment: - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local/envs/ofa

  added / updated specs:
    - python=3.7.4
    - pytorch=1.8.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _pytorch_select-0.1        |            cpu_0           3 KB
    blas-1.0                   |              mkl           6 KB
    certifi-2022.9.24          |   py37h06a4308_0         154 KB
    cffi-1.15.0                |   py37h7f8727e_0         222 KB
    flit-core-3.6.0            |     pyhd3eb1b0_0          42 KB
    intel-openmp-2019.4        |              243         729 KB
    libedit-3.1.20221030       |       h5eee18b_0         181 KB
    libffi-3.2.1               |    hf484d3e_1007          48 KB
    libmklml-2019.0.5 

There is an uresolved issue with conda that could affect our environment https://github.com/conda/conda/issues/9957 </br>
To address it, create a symlink for libffi.so.7 as a workaround if needed

In [ ]:
!ln -s /usr/local/envs/ofa/lib/libffi.so.6 /usr/local/envs/ofa/lib/libffi.so.7

Copy the OFA model checkpoints into the ./OFA/checkpoints/ in the current working directory </br>
You can find the checkpoints here https://drive.google.com/drive/folders/1zN8liXv8q3z-BV7Ipp1zcodkIqPY-E4G?usp=share_link
```
./
├── OFA/
│   ├── checkpoints/
│   │   ├── vqa_huge_best.pt
│   │   ├── checkpoint.ofa8160.pt
│   │   ├── checkpoint.ofa8150.pt
│   │   ├── checkpoint.6000_09.pt
│   │   ├── checkpoint.3000fulltrain.pt
│   │   └── checkpoint.nodrop8120.pt
│   └── ...
└── ...
```
In this example notebook we are copying these checkpoints from the mounted Google Drive 

Copy the CLIP model checkpoints into the ~~/.cache/huggingface/hub directory in the current user home directory </br>
You can find the checkpoints here https://drive.google.com/drive/folders/16Jqdq8o5IDdDk-pjW5M39294DkuN5Agz?usp=share_link
```
~/
├── .cache/
│   ├── huggingface/
│   │   ├── hub/
│   │   │   ├── models--xlm-roberta-large/
│   │   │   ├── models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/
│   │   │   └── version.txt
│   │   ├── ...
│   └── ...
└── ...
```
In this example notebook we are copying these checkpoints from the mounted Google Drive 

In [ ]:
!mkdir -p ~/.cache/huggingface/hub
!rsync -r --progress /content/gdrive/MyDrive/WSDM\ Toloka/frozen_laion5b_s13b_b90k/ ~/.cache/huggingface/hub

sending incremental file list
version.txt
              1 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=15/17)
models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/
models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/blobs/
models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/refs/
models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/refs/main
             40 100%    0.26kB/s    0:00:00 (xfr#2, to-chk=9/17)
models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/snapshots/
models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/snapshots/f9ab955287d06ac45c7654f26fecd29301c70a1f/
models--laion--CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k/snapshots/f9ab955287d06ac45c7654f26fecd29301c70a1f/open_clip_pytorch_model.bin
  4,772,387,963 100%  131.19MB/s    0:00:34 (xfr#3, to-chk=7/17)
models--xlm-roberta-large/
models--xlm-roberta-large/blobs/
models--xlm-roberta-large/refs/


Mount the requried directiories with input files in /mnt/data/test.csv and the input images at /mnt/data/imgs. </br>
In this example notebook we will put the public test data into this directories for a test run

In [ ]:
!mkdir -p /mnt/data/
!mkdir -p /mnt/output/

# Download the public test data from the Kaggle Datasets
!mkdir -p ~/.kaggle/
!cp /content/gdrive/MyDrive/.config/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle datasets download slavabarkov/toloka-visual-question-answering-dataset

# Move the data to the specified folders
!unzip -qq -n toloka-visual-question-answering-dataset.zip -d /mnt/data/
!mv /mnt/data/images /mnt/data/imgs
!mv /mnt/data/test_public.csv /mnt/data/test.csv

100% 6.10G/6.12G [00:50<00:00, 133MB/s]
100% 6.12G/6.12G [00:50<00:00, 131MB/s]


# Submission part: time starts

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
from torchvision import transforms
import open_clip

import json
import csv
from io import BytesIO
import base64

# 2. Getting VQA answers on questions

## 2.1 Converting csv dataset to tsv format for OFA

Input: dataframe with questions - `/mnt/data/test.csv`

Output: tsv file for OFA - `./OFA/dataset/vqa_data/vqa_test.tsv`

In [ ]:
# Function to convert images to base64

def get_string_image(img):
    img = Image.open(img)
    img_buffer = BytesIO()
    img.save(img_buffer, format=img.format)
    byte_data = img_buffer.getvalue()
    base64_str = base64.b64encode(byte_data) # bytes
    base64_str = base64_str.decode("utf-8") # str 
    return base64_str

In [ ]:
df = pd.read_csv('/mnt/data/test.csv')

In [ ]:
with open('./OFA/dataset/vqa_data/vqa_test.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')

    for i in tqdm(df.index.values):
        img = df.loc[i, 'image']
        img = img.split('/')[-1]
        img_id = img.split('.')[0]
        img = f'/mnt/data/imgs/{img}'
        image_base = get_string_image(img)
        
        text = df.loc[i, 'question']
        false_answer = "1.0|!+no"
        blank = ""
        
        writer.writerow([i, img_id, text, false_answer, blank, image_base])

100%|██████████| 1705/1705 [00:30<00:00, 55.51it/s]


## 2.2 Running OFA VQA on tsv dataset

In [ ]:
%cd ./OFA/run_scripts/vqa
!chmod +x ./evaluate_vqa_beam.sh
!source activate ofa && sh ./evaluate_vqa_beam.sh test

/content/OFA/run_scripts/vqa
/usr/local/envs/ofa/lib/python3.7/site-packages/torch/distributed/launch.py:188: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
2022-12-19 21:30:56 | INFO | torch.distributed.nn.jit.instantiator | Created a temporary directory at /tmp/tmpvscrtcmf
2022-12-19 21:30:56 | INFO | torch.distributed.nn.jit.instantiator | Writing /tmp/tmpvscrtcmf/_remote_module_non_scriptable.py
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping

In [ ]:
%cd ../../..

/content


## 2.3 Convert json predictions back to csv and add to the main dataframe

Input: json file with predictions from VQA - `./OFA/results/test_predict.json`

Output: csv dataframe with a new column "ofa_huge", located - `./ofa_vqa.csv`

In [ ]:
PATH = "./OFA/results/test_predict.json"

In [ ]:
df = pd.read_csv('/mnt/data/test.csv')

In [ ]:
with open(PATH, "r") as read_file:
    data = json.load(read_file)

In [ ]:
answers = [i["answer"] for i in data]
df["ofa_huge"] = answers

In [ ]:
df.to_csv("./ofa_vqa.csv", index=False)

# 3. Getting Visual Grounding predictions for "image - answer"

## 3.1 Convert new csv dataset to tsv

Input: csv dataframe with images/answers, located - `./ofa_vqa.csv`

Output: tsv file for OFA visual grounding, located - `/content/OFA/dataset/refcoco/test_vg.tsv`

In [ ]:
df = pd.read_csv("./ofa_vqa.csv")

In [ ]:
with open('./OFA/dataset/refcoco/test_vg.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')

    for i in tqdm(df.index.values):
        img = df.loc[i, 'image']
        img = img.split('/')[-1]
        img_id = img.split('.')[0]
        img = f'/mnt/data/imgs/{img}'
        image_base = get_string_image(img)
        
        text = df.loc[i, 'ofa_huge']
        false_answer = "0.0, 0.0, 0.0, 0.0"
        blank = ""
        
        writer.writerow([i, img_id, text, false_answer, image_base])

100%|██████████| 1705/1705 [00:33<00:00, 50.83it/s]


## 3.2 Run 5 models: OFA Visual Grounding on tsv dataset

Output: 5 files with predictions from visual grounding, file example - `./OFA/results/refcoco/vg_answer1.tsv_predict.json`


In [ ]:
%cd ./OFA/run_scripts/refcoco
!chmod +x ./evaluate_refcoco1.sh
!source activate ofa && sh ./evaluate_refcoco1.sh

/content/OFA/run_scripts/refcoco
/usr/local/envs/ofa/lib/python3.7/site-packages/torch/distributed/launch.py:188: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
2022-12-19 21:39:16 | INFO | torch.distributed.nn.jit.instantiator | Created a temporary directory at /tmp/tmpdblfsny8
2022-12-19 21:39:16 | INFO | torch.distributed.nn.jit.instantiator | Writing /tmp/tmpdblfsny8/_remote_module_non_scriptable.py
2022-12-19 21:39:20 | INFO | fairseq.distributed.utils | distributed init (rank 0): env://
2022-12-19 21:39:20 | INFO | fairseq.distributed.utils | Start init
2022-12-19 21:39:20 | INFO | torch.distributed.distributed_c10d | Added key: store_

In [ ]:
!chmod +x ./evaluate_refcoco2.sh
!source activate ofa && sh ./evaluate_refcoco2.sh

/usr/local/envs/ofa/lib/python3.7/site-packages/torch/distributed/launch.py:188: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
2022-12-19 21:42:39 | INFO | torch.distributed.nn.jit.instantiator | Created a temporary directory at /tmp/tmpaisdewwu
2022-12-19 21:42:39 | INFO | torch.distributed.nn.jit.instantiator | Writing /tmp/tmpaisdewwu/_remote_module_non_scriptable.py
2022-12-19 21:42:44 | INFO | fairseq.distributed.utils | distributed init (rank 0): env://
2022-12-19 21:42:44 | INFO | fairseq.distributed.utils | Start init
2022-12-19 21:42:44 | INFO | torch.distributed.distributed_c10d | Added key: store_based_barrier_key:1 to store for 

In [ ]:
!chmod +x ./evaluate_refcoco3.sh
!source activate ofa && sh ./evaluate_refcoco3.sh

/usr/local/envs/ofa/lib/python3.7/site-packages/torch/distributed/launch.py:188: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
2022-12-19 21:46:01 | INFO | torch.distributed.nn.jit.instantiator | Created a temporary directory at /tmp/tmp0ilbsaux
2022-12-19 21:46:01 | INFO | torch.distributed.nn.jit.instantiator | Writing /tmp/tmp0ilbsaux/_remote_module_non_scriptable.py
2022-12-19 21:46:05 | INFO | fairseq.distributed.utils | distributed init (rank 0): env://
2022-12-19 21:46:05 | INFO | fairseq.distributed.utils | Start init
2022-12-19 21:46:05 | INFO | torch.distributed.distributed_c10d | Added key: store_based_barrier_key:1 to store for 

In [ ]:
!chmod +x ./evaluate_refcoco4.sh
!source activate ofa && sh ./evaluate_refcoco4.sh

/usr/local/envs/ofa/lib/python3.7/site-packages/torch/distributed/launch.py:188: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
2022-12-19 21:49:28 | INFO | torch.distributed.nn.jit.instantiator | Created a temporary directory at /tmp/tmpd1r6j2dv
2022-12-19 21:49:28 | INFO | torch.distributed.nn.jit.instantiator | Writing /tmp/tmpd1r6j2dv/_remote_module_non_scriptable.py
2022-12-19 21:49:33 | INFO | fairseq.distributed.utils | distributed init (rank 0): env://
2022-12-19 21:49:33 | INFO | fairseq.distributed.utils | Start init
2022-12-19 21:49:33 | INFO | torch.distributed.distributed_c10d | Added key: store_based_barrier_key:1 to store for 

In [ ]:
!chmod +x ./evaluate_refcoco5.sh
!source activate ofa && sh ./evaluate_refcoco5.sh

/usr/local/envs/ofa/lib/python3.7/site-packages/torch/distributed/launch.py:188: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
2022-12-19 21:52:50 | INFO | torch.distributed.nn.jit.instantiator | Created a temporary directory at /tmp/tmpk5yuno0w
2022-12-19 21:52:50 | INFO | torch.distributed.nn.jit.instantiator | Writing /tmp/tmpk5yuno0w/_remote_module_non_scriptable.py
2022-12-19 21:52:55 | INFO | fairseq.distributed.utils | distributed init (rank 0): env://
2022-12-19 21:52:55 | INFO | fairseq.distributed.utils | Start init
2022-12-19 21:52:55 | INFO | torch.distributed.distributed_c10d | Added key: store_based_barrier_key:1 to store for 

In [ ]:
%cd ../../..

/content


## 3.3 Convert json prediction to csv dataset

Input: 5 paths to json predictions from visual grounding, file example - `./OFA/results/refcoco/vg_answer1.tsv_predict.json`


Output: 5 csv files, file example - `./data/ofa8160.csv `

In [ ]:
paths = [
         ('ofa8160', './OFA/results/refcoco/vg_answer1.tsv_predict.json'),
         ('ofa8150', './OFA/results/refcoco/vg_answer2.tsv_predict.json'),
         ('ofa6000', './OFA/results/refcoco/vg_answer3.tsv_predict.json'),
         ('ofa3000', './OFA/results/refcoco/vg_answer4.tsv_predict.json'),
         ('ofa8120', './OFA/results/refcoco/vg_answer5.tsv_predict.json')
        ]

In [ ]:
for name, path in paths:
    with open(path, "r") as read_file:
        data = json.load(read_file)

    # Get coordinates
    left = [i["box"][0] for i in data]
    top = [i["box"][1] for i in data]
    right = [i["box"][2] for i in data]
    bottom = [i["box"][3] for i in data]

    # Open dataframe
    df_test = pd.read_csv("/mnt/data/test.csv")
    # Save coordinates in it
    df_test["left"] = left
    df_test["right"] = right
    df_test["top"] = top
    df_test["bottom"] = bottom

    # Save the dataframe
    df_test.to_csv(f"./{name}.csv", index=False)

# 4. Ensembling models predictions

Inputs: 5 csv files, file example - `./data/ofa8160.csv`

Outputs: final predictions `/mnt/output/answer.csv`



In [ ]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
def load_answers(path):
    answers = pd.read_csv(path)

    answers['left'] = answers['left'].clip(lower=0, upper=answers['width'])
    answers['right'] = answers['right'].clip(lower=0, upper=answers['width'])
    answers['top'] = answers['top'].clip(lower=0, upper=answers['height'])
    answers['bottom'] = answers['bottom'].clip(lower=0, upper=answers['height'])

    for i in answers.index.values:
        if answers.loc[i, 'left'] > answers.loc[i, 'right']:
            answers.loc[i, 'left'], answers.loc[i, 'right'] = answers.loc[i, 'right'], answers.loc[i, 'left']
        if answers.loc[i, 'top'] > answers.loc[i, 'bottom']:
            answers.loc[i, 'top'], answers.loc[i, 'bottom'] = answers.loc[i, 'bottom'], answers.loc[i, 'top']
        if answers.loc[i, 'left'] == answers.loc[i, 'right']:
            answers.loc[i, 'right'] += 1
        if answers.loc[i, 'top'] == answers.loc[i, 'bottom']:
            answers.loc[i, 'bottom'] += 1
            
    return answers

In [ ]:
text_template = [
    lambda c: f'a bad photo of a {c}.',
    lambda c: f'a photo of many {c}.',
    lambda c: f'a sculpture of a {c}.',
    lambda c: f'a photo of the hard to see {c}.',
    lambda c: f'a low resolution photo of the {c}.',
    lambda c: f'a rendering of a {c}.',
    lambda c: f'graffiti of a {c}.',
    lambda c: f'a bad photo of the {c}.',
    lambda c: f'a cropped photo of the {c}.',
    lambda c: f'a tattoo of a {c}.',
    lambda c: f'the embroidered {c}.',
    lambda c: f'a photo of a hard to see {c}.',
    lambda c: f'a bright photo of a {c}.',
    lambda c: f'a photo of a clean {c}.',
    lambda c: f'a photo of a dirty {c}.',
    lambda c: f'a dark photo of the {c}.',
    lambda c: f'a drawing of a {c}.',
    lambda c: f'a photo of my {c}.',
    lambda c: f'the plastic {c}.',
    lambda c: f'a photo of the cool {c}.',
    lambda c: f'a close-up photo of a {c}.',
    lambda c: f'a black and white photo of the {c}.',
    lambda c: f'a painting of the {c}.',
    lambda c: f'a painting of a {c}.',
    lambda c: f'a pixelated photo of the {c}.',
    lambda c: f'a sculpture of the {c}.',
    lambda c: f'a bright photo of the {c}.',
    lambda c: f'a cropped photo of a {c}.',
    lambda c: f'a plastic {c}.',
    lambda c: f'a photo of the dirty {c}.',
    lambda c: f'a jpeg corrupted photo of a {c}.',
    lambda c: f'a blurry photo of the {c}.',
    lambda c: f'a photo of the {c}.',
    lambda c: f'a good photo of the {c}.',
    lambda c: f'a rendering of the {c}.',
    lambda c: f'a {c} in a video game.',
    lambda c: f'a photo of one {c}.',
    lambda c: f'a doodle of a {c}.',
    lambda c: f'a close-up photo of the {c}.',
    lambda c: f'a photo of a {c}.',
    lambda c: f'the origami {c}.',
    lambda c: f'the {c} in a video game.',
    lambda c: f'a sketch of a {c}.',
    lambda c: f'a doodle of the {c}.',
    lambda c: f'a origami {c}.',
    lambda c: f'a low resolution photo of a {c}.',
    lambda c: f'the toy {c}.',
    lambda c: f'a rendition of the {c}.',
    lambda c: f'a photo of the clean {c}.',
    lambda c: f'a photo of a large {c}.',
    lambda c: f'a rendition of a {c}.',
    lambda c: f'a photo of a nice {c}.',
    lambda c: f'a photo of a weird {c}.',
    lambda c: f'a blurry photo of a {c}.',
    lambda c: f'a cartoon {c}.',
    lambda c: f'art of a {c}.',
    lambda c: f'a sketch of the {c}.',
    lambda c: f'a embroidered {c}.',
    lambda c: f'a pixelated photo of a {c}.',
    lambda c: f'itap of the {c}.',
    lambda c: f'a jpeg corrupted photo of the {c}.',
    lambda c: f'a good photo of a {c}.',
    lambda c: f'a plushie {c}.',
    lambda c: f'a photo of the nice {c}.',
    lambda c: f'a photo of the small {c}.',
    lambda c: f'a photo of the weird {c}.',
    lambda c: f'the cartoon {c}.',
    lambda c: f'art of the {c}.',
    lambda c: f'a drawing of the {c}.',
    lambda c: f'a photo of the large {c}.',
    lambda c: f'a black and white photo of a {c}.',
    lambda c: f'the plushie {c}.',
    lambda c: f'a dark photo of a {c}.',
    lambda c: f'itap of a {c}.',
    lambda c: f'graffiti of the {c}.',
    lambda c: f'a toy {c}.',
    lambda c: f'itap of my {c}.',
    lambda c: f'a photo of a cool {c}.',
    lambda c: f'a photo of a small {c}.',
    lambda c: f'a tattoo of the {c}.',
]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_clip, _, preprocess_clip = open_clip.create_model_and_transforms('xlm-roberta-large-ViT-H-14', pretrained='frozen_laion5b_s13b_b90k')
tokenizer_clip = open_clip.get_tokenizer('xlm-roberta-large-ViT-H-14')
model_clip.to(device);

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
test_answers_ofa = [
                    load_answers('./ofa8160.csv'),
                    load_answers('./ofa3000.csv'),
                    load_answers('./ofa8150.csv'),
                    load_answers('./ofa6000.csv'),
                    load_answers('./ofa8120.csv')
                    ]

In [ ]:
test_answers = pd.read_csv('./ofa_vqa.csv')
test_answers.loc[:, ['left', 'top', 'right', 'bottom']] = 0.0

model_clip.eval()
for i in tqdm(test_answers.index.values):
    path = f'/mnt/data/imgs/{test_answers.loc[i, "image"].split("/")[-1]}'
    img = Image.open(path).convert('RGB')
    target_sizes = torch.Tensor([img.size[::-1]]).to(device)

    answer = test_answers.loc[i, "ofa_huge"]
    texts = [[t(answer) for t in text_template]]

    boxes_ofa = np.array([a.loc[i, ['left', 'top', 'right', 'bottom']].to_numpy() for a in test_answers_ofa])
    boxes_ofa_dict = [{'x1': b[0], 'y1': b[1], 'x2': b[2], 'y2': b[3]} for b in boxes_ofa]
    boxes_ofa_mean_all = []
    boxes_ofa_mean_flag = [True, ]

    for j in range(1, len(boxes_ofa_dict)):
        iou_models = get_iou(boxes_ofa_dict[0], boxes_ofa_dict[j])
        boxes_ofa_mean_flag.append(iou_models > 0.6)
    boxes_ofa_mean_all.append(np.mean(boxes_ofa[boxes_ofa_mean_flag], axis=0))

    boxes_ofa = boxes_ofa[[not f for f in boxes_ofa_mean_flag]]
    boxes_ofa_mean_flag = [True, ]
    if len(boxes_ofa) > 1:
        boxes_ofa_dict = [{'x1': b[0], 'y1': b[1], 'x2': b[2], 'y2': b[3]} for b in boxes_ofa]
        for j in range(1, len(boxes_ofa_dict)):
            iou_models = get_iou(boxes_ofa_dict[0], boxes_ofa_dict[j])
            boxes_ofa_mean_flag.append(iou_models > 0.6)
        boxes_ofa_mean_all.append(np.mean(boxes_ofa[boxes_ofa_mean_flag], axis=0))

    if len(boxes_ofa) > 0:
        boxes_ofa = boxes_ofa[[not f for f in boxes_ofa_mean_flag]]
        boxes_ofa_mean_all.extend(boxes_ofa)

    if len(boxes_ofa_mean_all) == 1:
        box_max_conf = boxes_ofa_mean_all[0]
    else:  
        # CLIP
        text_features = tokenizer_clip(texts[0])
        text_features = text_features.to(device)
        with torch.no_grad():
            text_features = model_clip.encode_text(text_features)
        text_features = torch.mean(text_features, 0)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        score_threshold = -torch.inf
        for box in boxes_ofa_mean_all:
            # Crop detected box
            img_crop = img.crop(box.tolist())

            # Crop to keep aspect ratio
            crop_side = max(*img_crop.size, 224)
            img_crop = transforms.functional.center_crop(img_crop, output_size=[crop_side, crop_side])
            img_crop = preprocess_clip(img_crop).unsqueeze(0)

            # Get CLIP embedding
            img_crop = img_crop.to(device)
            with torch.no_grad():
                image_features = model_clip.encode_image(img_crop)

            # Normalize
            image_features /= image_features.norm(dim=-1, keepdim=True)
            
            prob = torch.nn.functional.cosine_similarity(image_features, text_features)
            if prob > score_threshold:
                score_threshold = prob
                box_max_conf = box
            
    test_answers.loc[i, ['left', 'top', 'right', 'bottom']] = box_max_conf

test_answers['left'] = test_answers['left'].clip(lower=0, upper=test_answers['width'])
test_answers['right'] = test_answers['right'].clip(lower=0, upper=test_answers['width'])
test_answers['top'] = test_answers['top'].clip(lower=0, upper=test_answers['height'])
test_answers['bottom'] = test_answers['bottom'].clip(lower=0, upper=test_answers['height'])
test_answers.head()

100%|██████████| 1705/1705 [01:36<00:00, 17.74it/s]


,image,width,height,left,top,right,bottom,question,ofa_huge
0,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,500,375,127.075743,158.870374,214.627917,373.525562,Where can we cook meals,stove
1,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,480,640,349.010132,480.341473,446.859839,513.278448,what do we use to make and receive calls?,cell phone
2,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,500,375,320.491016,73.578908,488.591211,228.319852,Where can we watch news?,tv
3,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,426,127.198227,170.958527,378.070282,270.506683,What is used to change TV channels?,remote
4,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,472,0.000000,11.927563,101.406134,135.336525,What is used to carry more belongings?,backpack


In [ ]:
test_answers.loc[:, ['left', 'top', 'right', 'bottom']] = test_answers.loc[:, ['left', 'top', 'right', 'bottom']].apply(np.round)

In [ ]:
test_answers['left'] = test_answers['left'].clip(lower=0, upper=test_answers['width'])
test_answers['right'] = test_answers['right'].clip(lower=0, upper=test_answers['width'])
test_answers['top'] = test_answers['top'].clip(lower=0, upper=test_answers['height'])
test_answers['bottom'] = test_answers['bottom'].clip(lower=0, upper=test_answers['height'])

for i in test_answers.index.values:
    if test_answers.loc[i, 'left'] > test_answers.loc[i, 'right']:
        test_answers.loc[i, 'left'], test_answers.loc[i, 'right'] = test_answers.loc[i, 'right'], test_answers.loc[i, 'left']
    if test_answers.loc[i, 'top'] > test_answers.loc[i, 'bottom']:
        test_answers.loc[i, 'top'], test_answers.loc[i, 'bottom'] = test_answers.loc[i, 'bottom'], test_answers.loc[i, 'top']
    if test_answers.loc[i, 'left'] == test_answers.loc[i, 'right']:
        test_answers.loc[i, 'right'] += 1
    if test_answers.loc[i, 'top'] == test_answers.loc[i, 'bottom']:
        test_answers.loc[i, 'bottom'] += 1

In [ ]:
test_answers.loc[:, ['image', 'left', 'top', 'right', 'bottom']].to_csv('/mnt/output/answer.csv', index=False)

In [ ]:
!head -n 5 /mnt/output/answer.csv

image,left,top,right,bottom
https://toloka-cdn.azureedge.net/wsdmcup2023/000000001066.jpg,127.0,159.0,215.0,374.0
https://toloka-cdn.azureedge.net/wsdmcup2023/000000001255.jpg,349.0,480.0,447.0,513.0
https://toloka-cdn.azureedge.net/wsdmcup2023/000000001357.jpg,320.0,74.0,489.0,228.0
https://toloka-cdn.azureedge.net/wsdmcup2023/000000001474.jpg,127.0,171.0,378.0,271.0
